In [1]:
import pandas as pd
import ast
import numpy as np

In [2]:
curr_dir = ""

Load data from files

In [69]:

movie_list = pd.read_csv(curr_dir + 'Movie_list.csv',delimiter = ',',quotechar = '"')
movie_details = pd.read_csv(curr_dir + 'movie_details.csv',delimiter = ',',quotechar = '"').dropna(how='all').reset_index()
cast_details = pd.read_csv(curr_dir + 'cast_details.csv',delimiter = ',',quotechar = '"').dropna(how='all').reset_index()
crew_details = pd.read_csv(curr_dir + 'crew_details.csv',delimiter = ',',quotechar = '"',header=None,names=['adult', 'gender', 'id', 'known_for_department', 'name','original_name', 'popularity', 'profile_path','credit_id','department','job', 'movie_id','order']).dropna(how='all').reset_index()
collection_details = pd.read_csv(curr_dir + 'Collection_details.csv',delimiter = ',',quotechar = '"').dropna(how='all').reset_index()
genre_list = pd.read_csv(curr_dir + 'data_genre.csv',delimiter = ',',quotechar = '"').dropna(how='all').reset_index()

In [72]:
#Removing records with no target label
movie_details.drop(index=movie_details[movie_details.revenue == 0].index,inplace=True)
movie_details = movie_details.reset_index()

Feature Selection

In [73]:

features_movie_details = movie_details[['adult','belongs_to_collection','budget','genres','id','production_countries','release_date','revenue','runtime','original_language']]
features_cast_details = cast_details[['gender','known_for_department','popularity','movie_id','order']]
features_crew_details = crew_details[['gender','known_for_department','popularity','department','movie_id','order']]
features_collection_details = collection_details[['id','parts']]

Data Processing

In [74]:

temp_df = pd.DataFrame(columns = ['collection_popularity'])
index = 0
for parts in collection_details['parts']:
    temp = ast.literal_eval(parts)
    sum_popularity = 0
    parts = 0
    for part in temp:
        parts += 1
        sum_popularity += part['popularity']
    temp_df.loc[index] =  sum_popularity/parts
    index += 1

features_collection_details['collection_popularity'] = temp_df['collection_popularity']




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
features_movie_details.genres.unique()

AttributeError: ignored

In [81]:
features_movie_details = movie_details[['adult','belongs_to_collection','budget','genres','id','production_countries','release_date','revenue','runtime','original_language']]

In [ ]:
arr = np.zeros((7967,19))
temp_fs_df = pd.DataFrame(arr,columns = (genre_list.name.unique())).add_suffix('_genre')
for index,genres in enumerate(features_movie_details.genres):
  try:
    temp_genres = ast.literal_eval(genres)
    for temp_genre in temp_genres:
        attr = temp_genre['name'] + '_genre'
        temp_fs_df[attr].loc[index] = 1
        #print(temp_fs_df[temp_genre['name']].loc[index])
  except:
    print(genres)

features_movie_details = pd.concat([features_movie_details,temp_fs_df],axis = 1,join='inner')

In [54]:
cast_details.shape

(201491, 14)

In [77]:
arr = np.zeros((201491,4))
temp_fs_df = pd.DataFrame(arr,columns = (features_cast_details.gender.unique())).add_suffix('_gender')
for index,gender in enumerate(features_cast_details.gender):
    attr = str((gender)) + '_gender'
    temp_fs_df[attr].loc[index] = 1

features_cast_details = pd.concat([features_cast_details,temp_fs_df],axis = 1,join='inner')

In [56]:
crew_details.shape

(49122, 14)

In [78]:
arr = np.zeros((49122,4))
temp_fs_df = pd.DataFrame(arr,columns = (features_crew_details.gender.unique())).add_suffix('_gender')
for index,gender in enumerate(features_crew_details.gender):
    attr = str(int(gender)) + '_gender'
    temp_fs_df[attr].loc[index] = 1

features_crew_details = pd.concat([features_crew_details,temp_fs_df],axis = 1,join='inner')
features_cast_details.rename(columns={'0.0_gender':'0_gender','1.0_gender':'1_gender','2.0_gender':'2_gender','3.0_gender':'3_gender'},inplace= True)

In [83]:
for index,collection_info in enumerate(features_movie_details.belongs_to_collection):
    if (collection_info == collection_info):
        temp_info = ast.literal_eval(collection_info)
        features_movie_details['belongs_to_collection'].loc[index] = temp_info['id']


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


SyntaxError: ignored

In [ ]:
production_countries = []
for index,production_country in enumerate(movie_details.production_countries):
    temp_pcs = ast.literal_eval(production_country)
    for temp_pc in temp_pcs:
        if (temp_pc['name'] in production_countries):
            continue
        else:
            production_countries.append(temp_pc['name'])

len(production_countries)
#Due to high number of production countries, this field was left out of the analysis

112

Feature Creation

In [61]:
features_cast_details

,gender,known_for_department,popularity,movie_id,order,1_gender,2_gender,0_gender,3_gender,Acting_relevance
0,1,Acting,10.791,805411,1,1.0,0.0,0.0,0.0,1
1,1,Acting,10.989,805411,2,1.0,0.0,0.0,0.0,1
2,2,Acting,2.704,805411,3,0.0,1.0,0.0,0.0,1
3,2,Acting,4.656,805411,4,0.0,1.0,0.0,0.0,1
4,1,Acting,4.729,805411,5,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
201486,2,Acting,8.151,135595,0,0.0,1.0,0.0,0.0,1
201487,1,Acting,3.873,135595,1,1.0,0.0,0.0,0.0,1
201488,2,Acting,3.081,135595,2,0.0,1.0,0.0,0.0,1
201489,2,Acting,0.999,135595,3,0.0,1.0,0.0,0.0,1


In [62]:
features_cast_details[features_cast_details.known_for_department != 'Acting']

,gender,known_for_department,popularity,movie_id,order,1_gender,2_gender,0_gender,3_gender,Acting_relevance
179,2,Writing,2.068,354857,1,0.0,1.0,0.0,0.0,1
215,1,Directing,1.583,788210,1,1.0,0.0,0.0,0.0,1
273,2,Writing,0.600,490410,4,0.0,1.0,0.0,0.0,1
435,2,Sound,2.345,550385,2,0.0,1.0,0.0,0.0,1
461,2,Directing,4.084,1362,3,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
201162,0,Production,0.600,376453,2,0.0,0.0,1.0,0.0,1
201163,0,Writing,0.600,376453,3,0.0,0.0,1.0,0.0,1
201164,2,Camera,0.600,376453,4,0.0,1.0,0.0,0.0,1
201460,2,Directing,2.679,17665,4,0.0,1.0,0.0,0.0,1


In [85]:

features_cast_details['Acting_relevance'] = 1
features_cast_details[features_cast_details.known_for_department != 'Acting'] = 0.5

features_crew_details['Directing_relevance'] = 1
features_crew_details[features_crew_details.known_for_department != 'Directing'] = 0.5

Merge Datasets

In [86]:
features_movie_details = features_movie_details.add_suffix('_main')
features_cast_details = features_cast_details.add_suffix('_cast')
features_crew_details = features_crew_details.add_suffix('_crew')
features_collection_details = features_collection_details.add_suffix('_coll_details')

In [95]:
overall_Data.shape

(7889, 115)

In [76]:
features_movie_details.shape

(7967, 29)

Remove Duplicates

In [87]:
features_collection_details.drop_duplicates(subset=['id_coll_details'],keep='first',inplace= True)

In [88]:
overall_Data = pd.merge(features_movie_details,features_collection_details, left_on = 'belongs_to_collection_main',right_on = "id_coll_details",how="left",validate="many_to_one")

In [90]:
overall_Data = pd.merge(overall_Data, features_crew_details[features_crew_details.order_crew == 1.0],left_on = 'id_main',right_on = "movie_id_crew", suffixes = ('','_1'),how="left")
overall_Data = pd.merge(overall_Data, features_crew_details[features_crew_details.order_crew == 2.0],left_on = 'id_main',right_on = "movie_id_crew", suffixes = ('','_2'),how="left")
overall_Data = pd.merge(overall_Data, features_crew_details[features_crew_details.order_crew == 3.0],left_on = 'id_main',right_on = "movie_id_crew", suffixes = ('','_3'),how="left")

In [92]:
overall_Data = pd.merge(overall_Data, features_cast_details[features_cast_details.order_cast == 1.0],left_on = 'id_main', right_on = 'movie_id_cast', suffixes = ('','_1'),how="left")
overall_Data = pd.merge(overall_Data, features_cast_details[features_cast_details.order_cast == 2.0],left_on = 'id_main', right_on = 'movie_id_cast', suffixes = ('','_2'),how="left")
overall_Data = pd.merge(overall_Data, features_cast_details[features_cast_details.order_cast == 3.0],left_on = 'id_main', right_on = 'movie_id_cast', suffixes = ('','_3'),how="left")
overall_Data = pd.merge(overall_Data, features_cast_details[features_cast_details.order_cast == 4.0],left_on = 'id_main', right_on = 'movie_id_cast', suffixes = ('','_4'),how="left")
overall_Data = pd.merge(overall_Data, features_cast_details[features_cast_details.order_cast == 5.0],left_on = 'id_main', right_on = 'movie_id_cast', suffixes = ('','_5'),how="left")

In [94]:
overall_Data.drop_duplicates(subset=['id_main'],keep='first',inplace= True)

In [102]:
final_data = overall_Data[['adult_main','budget_main','id_main','release_date_main','revenue_main','runtime_main','original_language_main','Action_genre_main','Adventure_genre_main','Animation_genre_main','Comedy_genre_main','Crime_genre_main','Documentary_genre_main','Drama_genre_main','Family_genre_main','Fantasy_genre_main','History_genre_main','Horror_genre_main','Music_genre_main','Mystery_genre_main','Romance_genre_main','Science Fiction_genre_main','TV Movie_genre_main','Thriller_genre_main','War_genre_main','Western_genre_main','collection_popularity_coll_details','popularity_crew','0_gender_crew','1_gender_crew','2_gender_crew','3_gender_crew','Directing_relevance_crew','popularity_crew_2','0_gender_crew_2','1_gender_crew_2','2_gender_crew_2','3_gender_crew_2','Directing_relevance_crew_2','popularity_crew_3','0_gender_crew_3','1_gender_crew_3','2_gender_crew_3','3_gender_crew_3','Directing_relevance_crew_3','popularity_cast','2_gender_cast','1_gender_cast','0_gender_cast','3_gender_cast','Acting_relevance_cast','popularity_cast_2','2_gender_cast_2','1_gender_cast_2','0_gender_cast_2','3_gender_cast_2','Acting_relevance_cast_2','popularity_cast_3','2_gender_cast_3','1_gender_cast_3','0_gender_cast_3','3_gender_cast_3','Acting_relevance_cast_3','popularity_cast_4','2_gender_cast_4','1_gender_cast_4','0_gender_cast_4','3_gender_cast_4','Acting_relevance_cast_4','popularity_cast_5','2_gender_cast_5','1_gender_cast_5','0_gender_cast_5','3_gender_cast_5','Acting_relevance_cast_5']]

In [103]:
final_data.dropna(how = 'all',subset=['popularity_crew','0_gender_crew','1_gender_crew','2_gender_crew','3_gender_crew','Directing_relevance_crew','popularity_crew_2','0_gender_crew_2','1_gender_crew_2','2_gender_crew_2','3_gender_crew_2','Directing_relevance_crew_2','popularity_crew_3','0_gender_crew_3','1_gender_crew_3','2_gender_crew_3','3_gender_crew_3','Directing_relevance_crew_3','popularity_cast','2_gender_cast','1_gender_cast','0_gender_cast','3_gender_cast','Acting_relevance_cast','popularity_cast_2','2_gender_cast_2','1_gender_cast_2','0_gender_cast_2','3_gender_cast_2','Acting_relevance_cast_2','popularity_cast_3','2_gender_cast_3','1_gender_cast_3','0_gender_cast_3','3_gender_cast_3','Acting_relevance_cast_3','popularity_cast_4','2_gender_cast_4','1_gender_cast_4','0_gender_cast_4','3_gender_cast_4','Acting_relevance_cast_4','popularity_cast_5','2_gender_cast_5','1_gender_cast_5','0_gender_cast_5','3_gender_cast_5','Acting_relevance_cast_5'],inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [104]:
final_data.set_index('id_main')

,adult_main,budget_main,release_date_main,revenue_main,runtime_main,original_language_main,Action_genre_main,Adventure_genre_main,Animation_genre_main,Comedy_genre_main,Crime_genre_main,Documentary_genre_main,Drama_genre_main,Family_genre_main,Fantasy_genre_main,History_genre_main,Horror_genre_main,Music_genre_main,Mystery_genre_main,Romance_genre_main,Science Fiction_genre_main,TV Movie_genre_main,Thriller_genre_main,War_genre_main,Western_genre_main,collection_popularity_coll_details,popularity_crew,0_gender_crew,1_gender_crew,2_gender_crew,3_gender_crew,Directing_relevance_crew,popularity_crew_2,0_gender_crew_2,1_gender_crew_2,2_gender_crew_2,3_gender_crew_2,Directing_relevance_crew_2,popularity_crew_3,0_gender_crew_3,1_gender_crew_3,2_gender_crew_3,3_gender_crew_3,Directing_relevance_crew_3,popularity_cast,2_gender_cast,1_gender_cast,0_gender_cast,3_gender_cast,Acting_relevance_cast,popularity_cast_2,2_gender_cast_2,1_gender_cast_2,0_gender_cast_2,3_gender_cast_2,Acting_relevance_cast_2,popularity_cast_3,2_gender_cast_3,1_gender_cast_3,0_gender_cast_3,3_gender_cast_3,Acting_relevance_cast_3,popularity_cast_4,2_gender_cast_4,1_gender_cast_4,0_gender_cast_4,3_gender_cast_4,Acting_relevance_cast_4,popularity_cast_5,2_gender_cast_5,1_gender_cast_5,0_gender_cast_5,3_gender_cast_5,Acting_relevance_cast_5
id_main,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
523931.0,False,130178.0,2018-08-10,5303.0,90.0,en,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,33.365000,0.631,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600,1.0,0.0,0.0,0.0,1.0,0.600,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.600,0.0,1.0,0.0,0.0,1.0,1.452,1.0,0.0,0.0,0.0,1.0
13649.0,False,0.0,2007-08-17,7000000.0,111.0,en,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,27.062667,1.967,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.085,0.0,1.0,0.0,0.0,1.0,3.798,0.0,1.0,0.0,0.0,1.0,4.907,1.0,0.0,0.0,0.0,1.0,1.626,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
15691.0,False,5000000.0,1996-05-12,42000000.0,89.0,en,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,0.751,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.795,1.0,0.0,0.0,0.0,1.0,1.638,0.0,1.0,0.0,0.0,1.0,2.547,1.0,0.0,0.0,0.0,1.0,0.872,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
13001.0,False,7000000.0,2008-03-11,8900000.0,101.0,en,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,13.355000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.865,0.0,1.0,0.0,0.0,1.0,3.550,1.0,0.0,0.0,0.0,1.0,2.203,1.0,0.0,0.0,0.0,1.0,1.879,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
397520.0,False,0.0,2016-02-15,127345.0,85.0,en,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.375667,0.694,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.881,1.0,0.0,0.0,0.0,1.0,1.400,0.0,1.0,0.0,0.0,1.0,1.166,0.0,1.0,0.0,0.0,1.0,0.732,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12633.0,False,8000000.0,1983-08-05,5555305.0,105.0,en,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,2.805,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.367,1.0,0.0,0.0,0.0,1.0,1.555,1.0,0.0,0.0,0.0,1.0,3.044,0.0,1.0,0.0,0.0,1.0,2.663,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
277710.0,False,0.0,2016-01-08,32163.0,90.0,en,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.071,1.0,0.0,0.0,0.0,1.0,17.716,0.0,1.0,0.0,0.0,1.0,3.589,0.0,1.0,0.0,0.0,1

In [105]:
for str in ['popularity_crew','0_gender_crew','1_gender_crew','2_gender_crew','3_gender_crew','Directing_relevance_crew','popularity_crew_2','0_gender_crew_2','1_gender_crew_2','2_gender_crew_2','3_gender_crew_2','Directing_relevance_crew_2','popularity_crew_3','0_gender_crew_3','1_gender_crew_3','2_gender_crew_3','3_gender_crew_3','Directing_relevance_crew_3','popularity_cast','2_gender_cast','1_gender_cast','0_gender_cast','3_gender_cast','Acting_relevance_cast','popularity_cast_2','2_gender_cast_2','1_gender_cast_2','0_gender_cast_2','3_gender_cast_2','Acting_relevance_cast_2','popularity_cast_3','2_gender_cast_3','1_gender_cast_3','0_gender_cast_3','3_gender_cast_3','Acting_relevance_cast_3','popularity_cast_4','2_gender_cast_4','1_gender_cast_4','0_gender_cast_4','3_gender_cast_4','Acting_relevance_cast_4','popularity_cast_5','2_gender_cast_5','1_gender_cast_5','0_gender_cast_5','3_gender_cast_5','Acting_relevance_cast_5']:
    final_data[str].replace(np.nan,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
final_data = final_data[final_data.revenue_main != 0]

In [106]:
final_data['budget_main'].mean()

25128881.761959143

In [107]:
final_data[final_data.budget_main == 0]['budget_main'] = final_data['budget_main'].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
final_data.columns

Index(['adult_main', 'budget_main', 'id_main', 'release_date_main',
       'revenue_main', 'runtime_main', 'original_language_main',
       'Action_genre_main', 'Adventure_genre_main', 'Animation_genre_main',
       'Comedy_genre_main', 'Crime_genre_main', 'Documentary_genre_main',
       'Drama_genre_main', 'Family_genre_main', 'Fantasy_genre_main',
       'History_genre_main', 'Horror_genre_main', 'Music_genre_main',
       'Mystery_genre_main', 'Romance_genre_main',
       'Science Fiction_genre_main', 'TV Movie_genre_main',
       'Thriller_genre_main', 'War_genre_main', 'Western_genre_main',
       'collection_popularity_coll_details', 'popularity_crew',
       '0_gender_crew', '1_gender_crew', '2_gender_crew', '3_gender_crew',
       'Directing_relevance_crew', 'popularity_crew_2', '0_gender_crew_2',
       '1_gender_crew_2', '2_gender_crew_2', '3_gender_crew_2',
       'Directing_relevance_crew_2', 'popularity_crew_3', '0_gender_crew_3',
       '1_gender_crew_3', '2_gender_crew_

In [112]:
final_data = final_data.reindex(columns=['revenue_main','adult_main', 'budget_main', 'id_main', 'release_date_main',
       'runtime_main', 'original_language_main',
       'Action_genre_main', 'Adventure_genre_main', 'Animation_genre_main',
       'Comedy_genre_main', 'Crime_genre_main', 'Documentary_genre_main',
       'Drama_genre_main', 'Family_genre_main', 'Fantasy_genre_main',
       'History_genre_main', 'Horror_genre_main', 'Music_genre_main',
       'Mystery_genre_main', 'Romance_genre_main',
       'Science Fiction_genre_main', 'TV Movie_genre_main',
       'Thriller_genre_main', 'War_genre_main', 'Western_genre_main',
       'collection_popularity_coll_details', 'popularity_crew',
       '0_gender_crew', '1_gender_crew', '2_gender_crew', '3_gender_crew',
       'Directing_relevance_crew', 'popularity_crew_2', '0_gender_crew_2',
       '1_gender_crew_2', '2_gender_crew_2', '3_gender_crew_2',
       'Directing_relevance_crew_2', 'popularity_crew_3', '0_gender_crew_3',
       '1_gender_crew_3', '2_gender_crew_3', '3_gender_crew_3',
       'Directing_relevance_crew_3', 'popularity_cast', '2_gender_cast',
       '1_gender_cast', '0_gender_cast', '3_gender_cast',
       'Acting_relevance_cast', 'popularity_cast_2', '2_gender_cast_2',
       '1_gender_cast_2', '0_gender_cast_2', '3_gender_cast_2',
       'Acting_relevance_cast_2', 'popularity_cast_3', '2_gender_cast_3',
       '1_gender_cast_3', '0_gender_cast_3', '3_gender_cast_3',
       'Acting_relevance_cast_3', 'popularity_cast_4', '2_gender_cast_4',
       '1_gender_cast_4', '0_gender_cast_4', '3_gender_cast_4',
       'Acting_relevance_cast_4', 'popularity_cast_5', '2_gender_cast_5',
       '1_gender_cast_5', '0_gender_cast_5', '3_gender_cast_5',
       'Acting_relevance_cast_5'])

In [116]:
final_data.drop(labels=['original_language_main'],axis=1,inplace=True)

In [117]:
final_data.to_csv(curr_dir + 'final_data.csv')